In [1]:
from pathlib import Path
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
from rdkit import Chem
from rdkit.Chem import Descriptors, Draw, PandasTools

In [2]:
def calculate_ro5_properties(smiles):
    """
    Test if input molecule (SMILES) fulfills Lipinski's rule of five.

    Parameters
    ----------
    smiles : str
        SMILES for a molecule.

    Returns
    -------
    pandas.Series
        Molecular weight, number of hydrogen bond acceptors/donor and logP value
        and Lipinski's rule of five compliance for input molecule.
    """
    # RDKit molecule from SMILES
    molecule = Chem.MolFromSmiles(smiles)
    # Calculate Ro5-relevant chemical properties
    molecular_weight = Descriptors.ExactMolWt(molecule)
    n_hba = Descriptors.NumHAcceptors(molecule)
    n_hbd = Descriptors.NumHDonors(molecule)
    logp = Descriptors.MolLogP(molecule)
    # Check if Ro5 conditions fulfilled
    conditions = [molecular_weight <= 500, n_hba <= 10, n_hbd <= 5, logp <= 5]
    ro5_fulfilled = sum(conditions) >= 3
    # Return True if no more than one out of four conditions is violated
    return pd.Series(
        [molecular_weight, n_hba, n_hbd, logp, ro5_fulfilled],
        index=["molecular_weight", "n_hba", "n_hbd", "logp", "ro5_fulfilled"],
    )

In [4]:
molecules = pd.read_csv("Covid_data_lip_PIC50.csv")
print(molecules.shape)

test = molecules.astype(object) 
test.info()

(306, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Mol_BDB_ID           306 non-null    object
 1   Smiles               306 non-null    object
 2   IC50_nm              306 non-null    object
 3   bioactivity_class    306 non-null    object
 4   MW                   306 non-null    object
 5   LogP                 306 non-null    object
 6   NumHDonors           306 non-null    object
 7   NumHAcceptors        306 non-null    object
 8   NumHeteroatoms       306 non-null    object
 9   NumRotatableBonds    306 non-null    object
 10  NumValenceElectrons  306 non-null    object
 11  NumAromaticRings     306 non-null    object
 12  NumSaturatedRings    306 non-null    object
 13  NumAliphaticRings    306 non-null    object
 14  pIC50                306 non-null    object
dtypes: object(15)
memory usage: 36.0+ KB


In [7]:
# CHECK_OUTPUT
for name, smiles in zip(molecules["Mol_BDB_ID"], molecules["Smiles"]):
    print(f"Ro5 fulfilled for {name}: {calculate_ro5_properties(smiles)['ro5_fulfilled']}")

Ro5 fulfilled for 1074441: True
Ro5 fulfilled for 1074558: True
Ro5 fulfilled for 1074516: True
Ro5 fulfilled for 1074541: True
Ro5 fulfilled for 1074478: True
Ro5 fulfilled for 1074475: True
Ro5 fulfilled for 1074546: True
Ro5 fulfilled for 1074537: True
Ro5 fulfilled for 1074540: True
Ro5 fulfilled for 1074513: True
Ro5 fulfilled for 1074447: True
Ro5 fulfilled for 1074559: True
Ro5 fulfilled for 1074515: True
Ro5 fulfilled for 1074536: True
Ro5 fulfilled for 1074438: True
Ro5 fulfilled for 1074563: True
Ro5 fulfilled for 1074440: True
Ro5 fulfilled for 1074545: True
Ro5 fulfilled for 1074539: True
Ro5 fulfilled for 1074512: True
Ro5 fulfilled for 1074444: True
Ro5 fulfilled for 1074543: True
Ro5 fulfilled for 1074560: True
Ro5 fulfilled for 1074550: True
Ro5 fulfilled for 1074535: True
Ro5 fulfilled for 1074551: True
Ro5 fulfilled for 1074561: True
Ro5 fulfilled for 1074555: True
Ro5 fulfilled for 1074557: True
Ro5 fulfilled for 1074505: True
Ro5 fulfilled for 1074497: True
Ro5 fulf

In [9]:
ro5_properties = molecules["Smiles"].apply(calculate_ro5_properties)
ro5_properties

,molecular_weight,n_hba,n_hbd,logp,ro5_fulfilled
0,531.109658,6,1,4.25408,True
1,441.120382,7,1,2.34980,True
2,505.115080,6,1,4.60730,True
3,505.115080,6,1,4.60730,True
4,523.105658,6,1,4.90290,True
...,...,...,...,...,...
301,600.294785,6,4,3.41810,True
302,572.321000,7,4,2.59860,True
303,426.044106,6,0,4.45432,True
304,381.176067,11,6,-2.06280,False


In [10]:
molecules = pd.concat([molecules, ro5_properties], axis=1)
molecules.to_csv('Covid_active_ro5.csv')

In [12]:
molecules_ro5_fulfilled = molecules[molecules["ro5_fulfilled"]]
molecules_ro5_violated = molecules[~molecules["ro5_fulfilled"]]

print(f"# Total compounds in the data set: {molecules.shape[0]}")
print(f"# compounds fulfilled role of 5 : {molecules_ro5_fulfilled.shape[0]}")
print(f"# compounds not compliant with the Ro5: {molecules_ro5_violated.shape[0]}")

# Total compounds in the data set: 306
# compounds fulfilled role of 5 : 299
# compounds not compliant with the Ro5: 7


In [14]:
def calculate_mean_std(dataframe):
    """
    Calculate the mean and standard deviation of a dataset.

    Parameters
    ----------
    dataframe : pd.DataFrame
        Properties (columns) for a set of items (rows).

    Returns
    -------
    pd.DataFrame
        Mean and standard deviation (columns) for different properties (rows).
    """
    # Generate descriptive statistics for property columns
    stats = dataframe.describe()
    # Transpose DataFrame (statistical measures = columns)
    stats = stats.T
    # Select mean and standard deviation
    stats = stats[["mean", "std"]]
    return stats

In [15]:
molecules_ro5_fulfilled_stats = calculate_mean_std(
    molecules_ro5_fulfilled[["molecular_weight", "n_hba", "n_hbd", "logp"]]
)
molecules_ro5_fulfilled_stats

,mean,std
molecular_weight,457.539307,98.183315
n_hba,5.578595,1.376825
n_hbd,2.163880,1.675895
logp,2.713353,1.353642


In [16]:
molecules_ro5_violated_stats = calculate_mean_std(
    molecules_ro5_violated[["molecular_weight", "n_hba", "n_hbd", "logp"]]
)
molecules_ro5_violated_stats

,mean,std
molecular_weight,581.971551,146.248272
n_hba,9.571429,4.859943
n_hbd,6.142857,1.069045
logp,-0.157937,2.486295


In [17]:
def _scale_by_thresholds(stats, thresholds, scaled_threshold):
    """
    Scale values for different properties that have each an individually defined threshold.

    Parameters
    ----------
    stats : pd.DataFrame
        Dataframe with "mean" and "std" (columns) for each physicochemical property (rows).
    thresholds : dict of str: int
        Thresholds defined for each property.
    scaled_threshold : int or float
        Scaled thresholds across all properties.

    Returns
    -------
    pd.DataFrame
        DataFrame with scaled means and standard deviations for each physiochemical property.
    """
    # Raise error if scaling keys and data_stats indicies are not matching
    for property_name in stats.index:
        if property_name not in thresholds.keys():
            raise KeyError(f"Add property '{property_name}' to scaling variable.")
    # Scale property data
    stats_scaled = stats.apply(lambda x: x / thresholds[x.name] * scaled_threshold, axis=1)
    return stats_scaled